In [2]:
# See https://github.com/nitarshan/bayes-by-backprop/blob/master/Weight%20Uncertainty%20in%20Neural%20Networks.ipynb
# The network structure is basically copied, while the learning code has been written mostly by me

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import transforms
import matplotlib.pyplot as plt
import itertools
import importlib

from training.util import GaussianMixture
import training.bbb as bbb

In [39]:
importlib.reload(bbb)

<module 'training.bbb' from 'd:\\Uni\\Bachelorarbeit\\VariationalBayesianDL\\training\\bbb.py'>

In [24]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    print("No cuda device available; using the CPU")
    device = torch.device("cpu")

In [ ]:
# Train on MNIST

import training.mnist

batch_size = 5
trainloader = training.mnist.flattened_trainloader(batch_size)

pi = 0.5 # 0.25, 0.5, 0.75
sigma1 = np.exp(-1) # 0, 1, 2
sigma2 = np.exp(-7) # 6, 7, 8
prior = GaussianMixture(pi, sigma1, sigma2)

model = nn.Sequential(
    bbb.BayesianLinearLayer(28*28, 400, prior, prior, device, weight_draw="minibatch"), 
    nn.ReLU(), 
    bbb.BayesianLinearLayer(400, 400, prior, prior, device, weight_draw="minibatch"), 
    nn.ReLU(), 
    bbb.BayesianLinearLayer(400, 10, prior, prior, device, weight_draw="minibatch"), 
    nn.LogSoftmax(dim=1)
)
model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
loss_fn = torch.nn.NLLLoss(reduction="sum")
for epoch in range(1):
    loss = bbb.run_bbb_epoch(model, optimizer, loss_fn, trainloader, device)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}: loss {loss / (len(trainloader) * batch_size)}")
print(f"Final loss {loss / (len(trainloader) * batch_size)}")

In [ ]:
with torch.no_grad():
    net.eval()
    corrects = 0
    samples = 0
    for data, target in testloader:
        data = torch.flatten(data, start_dim=1).to(device)
        target = target.to(device)
        outputs = net(data)
        preds = torch.argmax(outputs, dim=1)
        corrects += ((preds - target) == 0).sum()
        samples += len(data)
    print(f"Test accuracy: {corrects / samples}")

Test accuracy: 0.9185999631881714


In [ ]:
net2 = BayesianNetwork(prior, prior).to(device)
net2.load_state_dict(torch.load("models/bbb_mnist_adam.pth"))

with torch.no_grad():
    net2.train(False)
    corrects = 0
    samples = 0
    for data, target in testloader:
        data = torch.flatten(data, start_dim=1).to(device)
        target = target.to(device)
        
        outputs = net2(data)
        preds = torch.argmax(outputs, dim=1)
        corrects += ((preds - target) == 0).sum()
        samples += len(data)
    print(f"Test accuracy: {corrects / samples}")

Test accuracy: 0.9228999614715576
